In [1]:
import yfinance as yf
import pandas as pd
from pathlib import Path
import math 

In [2]:
result_folder = Path.cwd().parent / 'data'

symbols = pd.read_csv(result_folder / 'simple_screener_results.csv')['Ticker'].tolist()
symbols = pd.Series(symbols).unique()
backup = pd.read_csv(result_folder / 'simple_screener_results.csv')['Ticker']
if len(backup) > 0:
    backup.to_csv(result_folder / 'known_tickers.csv', index=False)
owned_tickers = pd.read_csv(result_folder / 'owned_tickers.csv')['Ticker'].tolist()

In [ ]:
df = pd.DataFrame(columns=[
    'Ticker', 
    'short_name', 
    'recommendation_score',
    'value_score',  
    'analyst_mean%', 
    'insider_buy%', 
    'sector', 
    'industry', 
    'country',
    'P/E (TTM)', 
    'ROA%',
])

for symbol in symbols: 
    ticker = yf.Ticker(symbol)
    try:
        analyst_mean = 0
        if 'mean' in ticker.analyst_price_targets and ticker.info['currentPrice']:
            analyst_mean = ticker.analyst_price_targets['mean'] / ticker.info['currentPrice'] - 1
        
        roa = ticker.info['returnOnAssets']*100
        pe = ticker.info['trailingPE']
        
        value_score = format(2.6 + math.log(roa+5) - math.log(pe+25), '.2f')
        
        insider_buy = 0
        if 'Shares' in ticker.insider_purchases.columns:
            insider_buy = format(ticker.insider_purchases.loc[ticker.insider_purchases.index[4], 'Shares'], '.2f')
            if insider_buy == '<NA>':
                insider_buy = 0

        recommendation_score = format(float(value_score) + analyst_mean * 1.25 + float(insider_buy) * 2.0, '.2f')

        df.loc[symbol] = [
            symbol, 
            ticker.info['shortName'], 
            recommendation_score,
            value_score,
            format(analyst_mean * 100, '.2f'), 
            insider_buy,
            ticker.info['sector'], 
            ticker.info['industry'], 
            ticker.info['country'],
            format(pe, '.2f') if isinstance(pe, (int, float)) else 'N/A', 
            format(roa, '.2f') if isinstance(roa, (int, float)) else 'N/A',
        ]
    except Exception as e:
        print(f"Error with {symbol}: {e}. Removed Ticker from list.")

df.to_csv(result_folder / 'simple_screener_results.csv', index=False)
display(df)

print(f'Got data for {len(df)} stocks')

,Ticker,short_name,recommendation_score,value_score,analyst_mean%,insider_buy%,sector,industry,country,P/E (TTM),ROA%
MATAS.CO,MATAS.CO,Matas A/S,1.04,0.79,20.16,0,Consumer Cyclical,Specialty Retail,Denmark,23.74,3.84
TRIFOR.CO,TRIFOR.CO,Trifork Group AG,1.07,0.75,25.58,0,Technology,Information Technology Services,Switzerland,14.65,1.92
QQ.L,QQ.L,QINETIQ GROUP PLC ORD 1P,1.41,1.23,14.20,0,Industrials,Aerospace & Defense,United Kingdom,17.37,6.89
RNMBY,RNMBY,Rheinmetall AG,0.24,0.24,0.00,0,Industrials,Aerospace & Defense,Germany,90.09,7.06
SAABF,SAABF,Saab AB,0.37,0.37,0.00,0,Industrials,Aerospace & Defense,Sweden,49.20,3.80
BCKIY,BCKIY,Babcock International Group Plc,0.96,0.96,0.00,0,Industrials,Engineering & Construction,United Kingdom,22.00,5.08
BAESY,BAESY,BAE Systems PLC,0.64,0.80,-12.77,0,Industrials,Aerospace & Defense,United Kingdom,26.46,4.40
IVSO.ST,IVSO.ST,INVISIO AB,1.32,1.32,0.00,0,Technology,Communication Equipment,Sweden,54.27,19.45
NSKFF,NSKFF,Kongsberg Gruppen ASA,0.66,0.66,0.00,0,Industrials,Aerospace & Defense,Norway,55.60,7.82
GMAB,GMAB,Genmab A/S,2.85,2.18,53.41,0,Healthcare,Biotechnology,Denmark,10.47,20.66


Got data for 44 stocks


### Single import

In [4]:
ticker = "AAPL"
print(f"P/E: {yf.Ticker(ticker).info['trailingPE']:.2f}")
print(f"ROA%: {yf.Ticker(ticker).info['returnOnAssets']*100:.2f}")

P/E: 32.72
ROA%: 23.81
